<a href="https://colab.research.google.com/github/Frorozcoloa/FinancIA/blob/main/Notebooks/0-2%20Train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers wandb torchmetrics lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import wandb
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import set_seed
from torch import nn

def SEED(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    set_seed(seed)


SEED(42)


# Configuration by model
NUM_VARAIBLES = 3
NUM_LABELS = 3
num_labels = NUM_LABELS * NUM_VARAIBLES
divice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from transformers import ( 
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_constant_schedule_with_warmup,
)

# Configuring the model
num_labels = NUM_LABELS * NUM_VARAIBLES
model_name = "pysentimiento/robertuito-sentiment-analysis"
auto_tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([9]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
values = auto_tokenizer.encode_plus("Hola mundo", return_tensors="pt")

In [70]:
torch.argmax(logists, dim=0)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [67]:
logists = model(values["input_ids"], values["attention_mask"])

In [57]:
class FinanciaSentimental(Dataset):
    """This class is used to load the data and tokenize it"""
    def __init__(self, tokenizer, dataframe, columns, max_len=512):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        ## Columns to target
        self._columns = columns
        self.max_len = max_len
    
    @property
    def columns(self):
        """Return the columns to target"""
        return self._columns

    def __len__(self):
        """Return the length of the dataset"""
        return len(self.dataframe)
        
    def __getitem__(self, index):
        """Get the data at the index"""
        values = self.dataframe.iloc[index]
        text = values['text']
        label = values[self._columns].values.astype(np.float32)
        inputs = self.tokenizer.encode_plus(text, max_length=130, pad_to_max_length=True, padding='max_length', truncation=True, return_tensors='pt')
        label = torch.tensor(label, dtype=torch.float)
        input_ids = inputs["input_ids"].squeeze().to(dtype=torch.long)
        attention_mask = inputs["attention_mask"].squeeze().to(dtype=torch.long)
        
        inputs_dict = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label":label
        }

        return inputs_dict

In [85]:
from torchmetrics import Accuracy, F1Score, Precision, Recall
import lightning.pytorch as pl

class FinanciaMultilabel(pl.LightningModule):
    """This class is used to create the model"""
    def __init__(self, num_labels, model_name, class_weights):
        super().__init__()
        self.num_labels = num_labels
        # The models is multi-label, so we need to use BCEWithLogitsLoss
        self.loss = nn.BCEWithLogitsLoss(pos_weight=class_weights, reduction='none')
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels, ignore_mismatched_sizes=True)
        self._tokenizer = AutoTokenizer.from_pretrained(model_name)
        #Metrics for the model for training
        self.train_f1 = F1Score(task = "multilabel", num_labels=num_labels)
        self.train_accuracy = Accuracy(task ="multilabel", num_labels=num_labels)
        self.train_precision = Precision(task ="multilabel", num_labels=num_labels)
        self.train_recall = Recall(task ="multilabel", num_labels=num_labels)
        # Metrics for the model for validation
        self.val_f1 = F1Score(task = "multilabel", num_labels=num_labels)
        self.val_accuracy = Accuracy(task ="multilabel", num_labels=num_labels)
        self.val_precision = Precision(task ="multilabel", num_labels=num_labels)
        self.val_recall = Recall(task ="multilabel", num_labels=num_labels)
        
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask):
        """This function is used to forward the data through the model"""
       
        output = self.model(input_ids, attention_mask)
        logits  = output.logits 
        print(logits)
        return logits

    def training_step(self, batch, batch_idx):
        """This function is used to train the model"""
        labels = batch["label"]
        attention_mask = batch["attention_mask"]
        inputs_id = batch["input_ids"]
        logits = self(inputs_id, attention_mask)
        train_loss = self.loss(logits, labels)
        self.train_accuracy(logits, labels)
        self.train_f1(logits, labels)
        self.train_precision(logits, labels)
        self.train_recall(logits, labels)
        self.log("train/loss", train_loss.mean(), on_step=False, on_epoch=True)
        self.log("train/accuracy", self.train_accuracy, on_step=False, on_epoch=True)
        self.log("train/precision", self.train_precision, on_step=False, on_epoch=True)
        self.log("train/recall", self.train_recall, on_step=False, on_epoch=True)
        self.log("train/f1_score", self.train_f1_score, on_step=False, on_epoch=True)
        return train_loss

    def validation_step(self, batch, batch_idx):
        """This function is used to validate the model"""
        labels = batch["label"]
        attention_mask = batch["attention_mask"]
        inputs_id = batch["input_ids"]
        logits = self(inputs_id, attention_mask)
        val_loss = self.loss(logits, labels)
        self.val_accuracy(logits, labels)
        self.val_f1(logits, labels)
        self.val_recall(logits, labels)
        self.val_precision(logits, labels)
        
        self.log("val/loss", val_loss.mean(), on_step=False, on_epoch=True)
        self.log("val/accuracy", self.val_accuracy, on_step=False, on_epoch=True)
        self.log("val/precision", self.val_precision, on_step=False, on_epoch=True)
        self.log("val/recall", self.val_recall, on_step=False, on_epoch=True)
        self.log("val/f1_score", self.val_f1_score, on_step=False, on_epoch=True)
        self.log("val/loss", val_loss, on_step=False, on_epoch=True)
        return val_loss

    def configure_optimizers(self):
        """This function is used to configure the optimizer"""
        optimizer = torch.optim.AdamW(self.parameters())


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
train_df = "/content/drive/Shareddrives/Redes neuronales/Datasets/df_with_sentiment.csv"
df = pd.read_csv(train_df)
df = df[["id",	"text",	"target",	"target_sentiment",	"companies_sentiment",	"consumers_sentiment", "tag"]]
df = pd.get_dummies(df, columns = ["target_sentiment",	"companies_sentiment","consumers_sentiment"])
df_train = df[df.tag == "train"]
df_test = df[df.tag == "test"]
df_valid, df_test = train_test_split(df_test, test_size=0.5)

In [11]:
df_valid

,id,text,target,tag,target_sentiment_negative,target_sentiment_neutral,target_sentiment_positive,companies_sentiment_negative,companies_sentiment_neutral,companies_sentiment_positive,consumers_sentiment_negative,consumers_sentiment_neutral,consumers_sentiment_positive
820,80591,La UE advierte de las consecuencias económicas...,UE,test,1,0,0,1,0,0,1,0,0
775,72293,Empieza el juicio por los 500 seísmos que caus...,Castor,test,1,0,0,0,1,0,1,0,0
830,99766,Los ‘nuevos’ ERTE mantienen la prohibición de ...,ERTE,test,0,0,1,1,0,0,0,0,1
822,40165,Cellnex salta mañana al cuarto puesto por capi...,Cellnex,test,0,0,1,1,0,0,0,1,0
848,32667,"Stripe, el mayor 'unicornio' de Silicon Valley...",Stripe,test,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,7312,Así evoluciona el precio de la vivienda usada,precio de vivienda usada,test,0,1,0,0,1,0,0,1,0
842,25376,Santander aprueba la salida a Bolsa de Getnet ...,Santander,test,0,0,1,0,1,0,0,1,0
750,67465,Calviño aboga por retrasar la reforma fiscal h...,Calviño,test,0,0,1,0,1,0,1,0,0
828,310714,El consejo de Norwegian rechaza las dos oferta...,Norwegian,test,0,0,1,1,0,0,0,1,0


In [12]:
columns_varaibles = ["target_sentiment_negative",	"target_sentiment_neutral",	"target_sentiment_positive"	,"companies_sentiment_negative"	,"companies_sentiment_neutral",	"companies_sentiment_positive", 'consumers_sentiment_negative',
       'consumers_sentiment_neutral', 'consumers_sentiment_positive']

In [13]:
print(df.shape)
print(df_train.shape)

(905, 13)
(736, 13)


In [58]:
train_dataset = FinanciaSentimental(auto_tokenizer, df_train, columns_varaibles)
valid_dataset = FinanciaSentimental(auto_tokenizer, df_valid, columns_varaibles)
test_dataset = FinanciaSentimental(auto_tokenizer, df_test, columns_varaibles)

In [59]:
inputs = next(iter(train_dataset))
print(inputs.keys())

dict_keys(['input_ids', 'attention_mask', 'label'])


In [60]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [61]:
inputs = next(iter(train_dataloader))
print(inputs.keys())


dict_keys(['input_ids', 'attention_mask', 'label'])


In [86]:
class_weight = torch.ones(9)
model = FinanciaMultilabel(9, model_name,class_weight )

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([9]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(project='FinancIA', name='#1', save_code=True, log_model=False, sync_tensorboard=True, save_dir="./logs")
wandb_logger.watch(model, log="all")

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [87]:
from  lightning.pytorch import Trainer
from  lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from  lightning.pytorch.callbacks.progress import TQDMProgressBar
from  lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader, WeightedRandomSampler

checkpoint_callback = ModelCheckpoint(monitor="val/f1_score", mode="max", save_last=True, save_weights_only=True)
tqdm_callback = TQDMProgressBar(refresh_rate=1)
early_stop_callback = EarlyStopping(monitor='val_loss', patience=5)
trainer = pl.Trainer( accelerator="cuda", max_epochs=10, logger=wandb_logger, callbacks=[checkpoint_callback, tqdm_callback,early_stop_callback], precision=16,)
trainer.fit(model, train_dataloader, valid_dataloader)

/usr/local/lib/python3.9/dist-packages/lightning/fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.9/dist-packages/lightning/pytorch/core/optimizer.py:174: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(
INFO: 
  | Name            | Type 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


tensor([[-1.8689e-01,  4.2310e-01,  1.8604e-01,  1.6418e-02,  1.7920e-01,
         -7.4036e-02, -3.9087e-01,  2.5073e-01,  2.1155e-01],
        [-1.0938e-01,  3.3960e-01, -3.1403e-02,  4.0131e-02,  1.4526e-01,
         -6.4209e-02, -3.4863e-01,  3.2812e-01,  7.7820e-02],
        [-1.1945e-01,  4.8657e-01, -2.4341e-01,  6.8054e-02,  1.9360e-01,
         -3.5718e-01,  1.6895e-01,  6.2012e-02,  1.9336e-01],
        [-4.5288e-02,  4.1797e-01, -3.2764e-01, -5.6419e-03,  2.8491e-01,
         -9.6985e-02, -2.9150e-01,  1.4600e-01,  5.3406e-02],
        [ 1.7395e-01,  1.9836e-01, -1.1011e-01,  1.0596e-01,  1.6052e-01,
         -2.2815e-01, -1.0114e-01, -4.5074e-02, -2.2598e-02],
        [ 2.6978e-01,  3.5938e-01, -3.3008e-01,  7.8735e-02,  1.0315e-01,
         -4.9414e-01,  6.3354e-02,  1.9861e-01,  2.0447e-02],
        [ 2.1729e-01,  5.7812e-01, -3.5547e-01,  2.1436e-01, -1.3824e-02,
         -4.1473e-02, -9.2041e-02, -4.3237e-01, -4.3030e-02],
        [ 1.7700e-02,  1.4954e-01,  3.7750e-02, 

AttributeError: ignored